In [22]:
import pandas as pd 
import numpy as np 
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import cv2 as cv

In [23]:
df_train=pd.read_csv('mnist_train.csv')
df_test=pd.read_csv('mnist_test.csv')

In [24]:
X_train = df_train.iloc[:, 1:].values
Y_train = df_train.iloc[:, 0].values
X_test = df_test.iloc[:, 1:].values
Y_test = df_test.iloc[:, 0].values 

In [25]:
model=Sequential([
    Dense(units=128, activation='relu', input_shape=(784,)),  
    Dense(units=128, activation='relu'),
    Dense(units=10, activation='softmax')
])

/Users/jatinkushwaha/anaconda3/envs/venv-metal/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
t0 = time.time()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=3000)
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
target_accuracy=test_accuracy
t1 = time.time()
target_time = t1-t0

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4279 - loss: 39.4972
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8221 - loss: 4.0712
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8747 - loss: 2.2278
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8881 - loss: 1.6565
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9015 - loss: 1.2849
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9110 - loss: 1.0354
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9183 - loss: 0.8683
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9265 - loss: 0.7271
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9332 - loss: 0.6165
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9395 - loss: 0.5194
Epoch 11/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9425 - loss: 0.4840
Epoch 12/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy

In [27]:
X_train = X_train.T
X_test = X_test.T
X_train.shape

(784, 60000)

In [28]:
def initialise():
    W1 = np.random.randn(10, 784) * 0.01
    b1 = np.random.randn(10, 1) * 0.01
    W2 = np.random.randn(10, 10) * 0.01
    b2 = np.random.randn(10, 1) * 0.01
    return W1,b1,W2,b2

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims = True))
    return expZ / np.sum(expZ, axis=0, keepdims = True)

def ReLU(Z):
    return np.maximum(0,Z)

def der_relu(Z):
    return np.where(Z>0,1,0) 

def forward(W1, b1, W2, b2, X):
    Z1=np.dot(W1,X) + b1
    A1=ReLU(Z1)
    Z2=np.dot(W2,A1) + b2
    A2=softmax(Z2)
    return Z1,A1,Z2,A2

def one_hot(Y):
    zero_matrix = np.zeros((10, len(Y)))
    for i in range(len(Y)):
        zero_matrix[Y[i],i] = 1
    return zero_matrix

def backward(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    m=X.shape[1] 
    dW2=(1/m)*np.dot(dZ2,np.transpose(A1))
    db2=(1/m)*np.sum(dZ2)
    dZ1=np.dot(np.transpose(W2),dZ2)*der_relu(Z1)
    dW1=(1/m)* np.dot(dZ1,np.transpose(X))
    db1=(1/m)*np.sum(dZ1)
    return dW1, db1, dW2, db2

def update(dW1, db1, dW2, db2, W1, W2, b1, b2, alpha):
    W1-=alpha*dW1
    b1-=alpha*db1
    W2-=alpha*dW2
    b2-=alpha*db2
    return W1, b1, W2, b2

In [29]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def getloss(A2, Y):
    m = Y.shape[0]  
    one_hot_Y = one_hot(Y)  
    log_likelihood = -np.sum(one_hot_Y * np.log(A2 + 1e-8), axis=0)
    loss = np.sum(log_likelihood) / m
    return loss
    
def fit(X,Y,alpha,iter):
    W1,b1,W2,b2=initialise()
    for i in range(iter): 
        Z1,A1,Z2,A2=forward(W1,b1,W2,b2,X)
        dW1,db1,dW2,db2=backward(Z1,A1,Z2,A2,W1,W2,X,Y)
        W1,b1,W2,b2=update(dW1,db1,dW2,db2,W1,W2,b1,b2,alpha)
        if(i!=0):
            print("▅▅▅▅▅▅▅▅▅▅▅▅", end='')
        if i % 10 == 0:
            print("\nIteration: ", i)
            predictions = get_predictions(A2)
            print("Accuracy = ",get_accuracy(predictions, Y)," Loss = ",getloss(predictions,Y))
    print("▅▅▅▅▅▅▅▅▅▅▅▅", end='')
    print("\nIteration: ", iter)
    predictions = get_predictions(A2)
    print("Accuracy = ",get_accuracy(predictions, Y)," Loss = ",getloss(predictions,Y))
    return W1, b1, W2, b2

In [30]:
t0=time.time()
W1, b1, W2, b2=fit(X_train, Y_train, 0.001, 1000)
customNN_time=time.time()-t0


Iteration:  0
Accuracy =  0.08638333333333334  Loss =  -0.40348533534419984
▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Iteration:  10
Accuracy =  0.35155  Loss =  2.4559123737758783
▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Iteration:  20
Accuracy =  0.56305  Loss =  0.9876381255373726
▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Iteration:  30
Accuracy =  0.6910833333333334  Loss =  0.7181887368582229
▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Iteration:  40
Accuracy =  0.7521333333333333  Loss =  0.6431859714969247
▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Iteration:  50
Accuracy =  0.786183333333333

In [44]:
def evaluate(X,Y):
    _, _, _, A2=forward(W1,b1,W2,b2,X)
    predictions=get_predictions(A2)
    print(f"Target accuracy on new data: {target_accuracy*100:.2f}% - Custom NN current accuracy: {get_accuracy(predictions, Y)*100:.2f}%")
    print(f"Target time on new data: {target_time:.2f}s - Custom NN time: {customNN_time:.2f}s")

In [45]:
evaluate(X_test,Y_test)

Target accuracy on new data: 94.04% - Custom NN current accuracy: 92.17%
Target time on new data: 15.29s - Custom NN time: 248.74s
